In [1]:
# se cargan todas las librerias
# Este notebook se corrio en python 3.9.13
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd 
import json
import spacy
import re
import unicodedata
import inflect
import keras 
import tensorflow 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np 
from datasets import Dataset, Features, ClassLabel, Value, DatasetDict, Sequence 
from sklearn.preprocessing import MinMaxScaler 
import torch.nn as nn
from torch.nn import Linear 

d:\reto_ia\ia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('train.csv')
test_q = pd.read_csv('test_queries.csv')
test_d = pd.read_csv('test_documents.csv')

In [3]:
# Se crean diferentes funciones para la limpieza de texto.
def quitar_tildes(texto):
    """
    Elimina los tildes de una cadena de texto.

    Parámetros:
    - texto: Cadena de texto con tildes.

    Retorna:
    Una nueva cadena de texto sin tildes.
    """
    texto_sin_tildes = unidecode(texto)
    return texto_sin_tildes


# Cargar el modelo de Spacy para inglés
nlp = spacy.load("es_core_news_sm")

# Funcion para quitar caracteres especiales de un texto utilizando expresiones regulares
def remove_special_characters(text):
    # Aplicar el modelo de Spacy para dividir el texto en tokens
    doc = nlp(text)

    # Reconstruir el texto eliminando caracteres especiales
    cleaned_text = ' '.join([token.text for token in doc if not re.search('[^A-Za-z0-9]+', token.text)])

    return cleaned_text

def remove_hashtag_and_url(words):
    # Eliminar URLs
    words = re.sub(r'http\S+|www.\S+', '', words)
    # Eliminar hashtags
    words = re.sub(r'#\S+', '', words)
    # Eliminar usuarios @
    words = re.sub(r'@\S+', '', words)
    # Elimina palabras con mas de aa
    words =re.sub(r'\b\w*aa\w*\b','',words)
    # Elimina palabras con mas de hh
    words =re.sub(r'\b\w*hh\w*\b','',words)
    # Elimina palabras con mas un ja r'mm+'
    words =re.sub(r'(ja){2,}','',words)
    # Elimina palabras con mas un mm
    words =re.sub(r'mm+','',words)

    return words



def remove_non_ascii(words):
    """
    Esta función toma una lista de palabras y elimina los caracteres no ASCII de cada palabra.
    
    Parámetros:
    - words: Una lista de palabras que se van a limpiar.
    
    Retorna:
    - Una cadena de texto que contiene las palabras limpias, sin caracteres no ASCII.
    """
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_word = new_word.replace('_', '')
        new_word = new_word.replace('\n', '')
        new_words.append(new_word)
    return str.join('', new_words)


def reduce_repeated_characters(word, max_repeats=2):
    # Utilizar una expresión regular para encontrar repeticiones excesivas
    pattern = re.compile(r'(\w)\1{%d,}' % (max_repeats - 1))

    # Función de sustitución para reducir repeticiones
    def reduce_match(match):
        return match.group(1) * max_repeats

    # Aplicar la función de sustitución a la palabra
    reduced_word = pattern.sub(reduce_match, word)

    return reduced_word


def to_lowercase(words):
    #Convert all characters to lowercase from list of tokenized words
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return str.join('',new_words)

def remove_punctuation(text):
    # Remove punctuation from text using regular expression
    text_without_punctuation = re.sub(r'[^\w\s]', '', text)
    return text_without_punctuation


# Inicializar el objeto de la clase Inflect
p = inflect.engine()

# Función para quitar numeros

def remove_numbers(text):
    words = []
    for word in text.split():
        # Verificar si la palabra contiene solo dígitos
        if not re.match(r'^\d+$', word):
            words.append(word)
    return ' '.join(words)


def remove_stopwords_spacy(text):
    """
    Stopwords de la libreria de spacy

    """
    doc = nlp(text)
    palabras_sin_stopwords = [token.text for token in doc if not token.is_stop]
    return ' '.join(palabras_sin_stopwords)

mis_stopwords = {"a", "al", "algo", "algunas", "algunos", "ante", "antes", "como", "con", "contra",
        "cual", "cualquier", "cualquieras", "cuales", "cualesquiera", "cuando", "de", "deja",
        "dejais", "dejamos", "dejan", "dejar", "del", "demas", "demasiada", "demasiadas",
        "demasiado", "demasiados", "dentro", "donde", "durante", "e", "el", "ella", "ellas",
        "ellos", "en", "entre", "era", "erais", "eramos", "eran", "eras", "eres", "es", "esa",
        "esas", "ese", "eso", "esos", "esta", "estaba", "estabais", "estábamos", "estaban",
        "estabas", "estad", "estada", "estadas", "estado", "estados", "estamos", "estando",
        "estar", "estara", "estaran", "estaras", "estare", "estareis", "estaremos", "estaría",
        "estaríais", "estaríamos", "estarian", "estarias", "estas", "este", "esto", "estos",
        "estoy", "estuve", "estuviera", "estuvierais", "estuvieran", "estuvieras", "estuvieron",
        "estuviese", "estuvieseis", "estuviesemos", "estuviesen", "estuvieses", "estuvimos",
        "estuviste", "estuvisteis", "estuvo", "está", "estabamos", "estais", "están", "estás",
        "fue", "fuera", "fuerais", "fueran", "fueras", "fueron", "fuese", "fueseis", "fuésemos",
        "fuesen", "fueses", "fui", "fuimos", "fuiste", "fuisteis", "ha", "habida", "habidas",
        "habido", "habidos", "habiendo", "habremos", "habrá", "habrán", "habrás", "habré",
        "habreis", "habremos", "habría", "habríais", "habríamos", "habrian", "habrias", "habeis",
        "había", "habíais", "habíamos", "habían", "habías", "han", "has", "hasta", "hay", "haya",
        "hayamos", "hayan", "hayas", "hayáis", "he", "hemos", "hube", "hubiera", "hubierais",
        "hubieran", "hubieras", "hubieron", "hubiese", "hubieseis", "hubiesemos", "hubiesen",
        "hubieses", "hubimos", "hubiste", "hubisteis", "hubo", "la", "las", "le", "les", "lo", "los",
        "me", "mi", "mis", "mucho", "muchos", "muy", "mas", "mi", "mia", "mias", "mio", "mios", "nada",
        "ni", "no", "nos", "nosotras", "nosotros", "nuestra", "nuestras", "nuestro", "nuestros", "o",
        "os", "otra", "otras", "otro", "otros", "para", "pero", "poco", "por", "porque", "que",
        "quien", "quienes", "qué", "se", "sea", "seamos", "sean", "seas", "sentid", "sentida",
        "sentidas", "sentido", "sentidos", "seremos", "sera", "seran", "seras", "seré", "seréis",
        "seremos", "seria", "seriais", "seríamos", "serían", "serías", "seáis", "siente", "sin",
        "sintiendo", "sobre", "sois", "somos", "son", "soy", "su", "sus", "suya", "suyas", "suyo",
        "suyos", "si", "tambien", "tanto", "te", "tendremos", "tendra", "tendran", "tendras", "tendre",
        "tendréis", "tendríamos", "tendría", "tendríais", "tendríamos", "tendrían", "tendrías",
        "tened", "tenemos", "tenga", "tengamos", "tengan", "tengas", "tengo", "tengais", "tenida",
        "tenidas", "tenido", "tenidos", "teniendo", "teneis", "tenia", "teniais", "teniamos", "tenian",
        "tenias", "ti", "tiene", "tienen", "tienes", "todo", "todos", "tu", "tus", "tuve", "tuviera",
        "tuvierais", "tuvieran", "tuvieras", "tuvieron", "tuviese", "tuvieseis", "tuviésemos", "tuviesen",
        "tuvieses", "tuvimos", "tuviste", "tuvisteis", "tuvo", "tuya", "tuyas", "tuyo", "tuyos", "tú",
        "un", "una", "uno", "unos", "vosotras", "vosotros", "vuestra", "vuestras", "vuestro", "vuestros",
        "y", "ya", "yo", "él", "eramos"}

# Función para quitar stopwords con spaCy, incluyendo las stopwords personalizadas
def remove_stopwords_spacy(text):
    for palabra in mis_stopwords:
        nlp.Defaults.stop_words.add(palabra)

    doc = nlp(text)
    palabras_sin_stopwords = [token.text for token in doc if not token.is_stop]

    # Verificar si la lista de palabras sin stopwords está vacía
    if palabras_sin_stopwords:
        return ' '.join(palabras_sin_stopwords)
    else:
        return text


def lemmatize_verbs(text):
    lemmatized_tokens = []
    doc = nlp(text)
    for token in doc:
        try:
            lemma = token.lemma_
            if lemma:
                lemmatized_tokens.append(lemma)
            else:
                lemmatized_tokens.append(token.text)  # Agregar la palabra original si no hay lema
        except Exception as e:
            print(f"Error al lematizar el token '{token}': {e}")
            lemmatized_tokens.append(token.text)  # Agregar la palabra original en caso de error

    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text 

def normalizador(df,columna):
    df['text2'] = df[columna].apply(quitar_tildes)
    df['text2'] = df['text2'].apply(remove_punctuation)
    df['text2'] = df['text2'].apply(remove_special_characters)
    df['text2'] = df['text2'].apply(remove_non_ascii)
    df['text2'] = df['text2'].apply(reduce_repeated_characters)
    df['text2'] = df['text2'].apply(to_lowercase)
    df['text2'] = df['text2'].apply(remove_numbers)
    df['text2'] = df['text2'].apply(remove_stopwords_spacy)
    df['text2'] = df['text2'].apply(lemmatize_verbs)
    return df['text2']


#train_document = normalizador(train, 'document')
#train_query =  normalizador(train, 'query')

#test_document = normalizador(test_d, 'document')
#test_query  = normalizador(test_q, 'query')

train_document = pd.read_csv('train_document_vector.csv')
train_query = pd.read_csv('train_query_vector.csv')
train_label = pd.read_csv('train_label_vector.csv')

test_document = pd.read_csv('test_document_vector.csv')
test_query = pd.read_csv('test_query_vector.csv')


In [53]:
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer, AdamW
from torch.nn import BCEWithLogitsLoss


modelo = 'distilbert-base-uncased'

# DistilBERT es una versión más pequeña y rápida de BERT que conserva gran parte de su rendimiento. El modelo está diseñado para tareas de clasificación de texto.
# Es un modelo de atención basado en transformadores.
# La capa de embeddings maneja la representación de palabras en vectores de alta dimensión. Utiliza embeddings de palabras y embeddings de posición para capturar la información semántica y contextual.
# Utiliza capas lineales o densas para calcular los pesos mas un sesgo
# Tiene una capa MultiHeadSelfAttention, esta capa permite que el modelo preste atención a diferentes partes de la secuencia de entrada de manera simultánea y en paralelo
# esto permite que el modelo capture patrones complejos y relaciones a diferentes escalas en la secuencia de entrada.
# modelo = 'roberta-base' # es posible utilizar roberta-base pero este modelo es mucho mas grande y llevara mas timepo entrenarlo.
# 

# se define un label que represente si es una mentira o una verdad
label1= {0: "Norelevant", 1:"Relevant"}
label2 = {"Norelevant":0, "Relevant":1}

# generamos el modelo de clasificacion para distilbert_base_uncased 
model = AutoModelForSequenceClassification.from_pretrained(
    modelo, num_labels=2, id2label=label1, label2id=label2)
model 


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [7]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

train_query['text2'].apply(lambda x: len(word_tokenize(x))).max() # se determina la maxima longitud de caracteres en query

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pulzara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


8

In [8]:
test_document['text2'].apply(lambda x: len(word_tokenize(x))).max()  # se determina la maxima longitud de caracteres en query

118

In [9]:
# create tokenizer
# Se carga un tokenizador preentrenado y se asegura que tenga un espacio antes del primer token.
# Esto se utiliza comunmente para el procesamiento de secuencias en el lenguaje natural (NLP).
tokenizer = AutoTokenizer.from_pretrained(modelo, add_prefix_space=True) 

def tokenize_and_pad(df, text_column, tokenizer):
    sequences = tokenizer(df[text_column].tolist(), return_tensors='np', padding=True, truncation=True, max_length = 118)
    return sequences['input_ids'], sequences['attention_mask']


In [10]:
input_train_queries, attention_mask_train_queries= tokenize_and_pad(train_query, 'text2', tokenizer=tokenizer)
input_train_documents, attention_mask_train_documents= tokenize_and_pad(train_document, 'text2', tokenizer=tokenizer)

input_test_queries, attention_mask_test_queries = tokenize_and_pad(test_query, 'text2', tokenizer=tokenizer)
input_test_documents, attention_mask_test_documents = tokenize_and_pad(test_document, 'text2', tokenizer=tokenizer) 

En la siguiente celda se suma el tensor de input de query con el input de documents y lo mismo para attention_mask con el fin de juntar la informacion y ser procesada por el modelo para que aprenda de ambos y clasifique a partir de esta nueva estructura. 

In [11]:
input_train = np.hstack([input_train_queries, input_train_documents])
attetntion_mask_train =  np.hstack([attention_mask_train_queries, attention_mask_train_documents])

In [12]:
text_train = train['query'] + ' ' + train['document']

In [13]:
# Se definen las variable spara construit el Dataset de entrenamiento
labels = train['label']

train_tokenized = pd.DataFrame(text_train).rename(columns={0:'text'}) 
train_tokenized['input_ids'] = list(input_train_queries)  
train_tokenized['attention_mask'] = list(attention_mask_train_queries)
train_tokenized['label'] = list(labels) 

# Renombramos la columna text 
train_tokenized = train_tokenized[['label','text','input_ids','attention_mask']]

In [14]:
# Definir las características
from datasets import Dataset, Features, ClassLabel, Value, DatasetDict, Sequence 

features = Features({
    'label': ClassLabel(names=[0, 1], id=None),
    'text': Value(dtype='string', id=None),
    'input_ids': Sequence(feature=Value(dtype='int64', id=None)),
    'attention_mask': Sequence(feature=Value(dtype='int64', id=None)) 
})  

train_tokenized.reset_index(drop=True, inplace=True)

# Crear el conjunto de datos, en este caso no dividimos la data si no que vamos a realizar la validacion con los mismos datos de entrenamiento para entrenar el modelo
train_dataset = Dataset.from_pandas(train_tokenized, features=features)  
test_dataset = Dataset.from_pandas(train_tokenized, features=features)  

In [ ]:
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
})

In [16]:
dataset_dict 

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
})

In [17]:
# Se crea un recopilador de datos, que rellenara dinámicamente los ejemplos en cada lote durante el entrenamiento 
# de modo que todos tengan la misma longitud. Esto es computacionalmente más eficiente que rellenar todos los ejemplos para que tengan la misma longitud 
# en todo el conjunto de datos.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# Metrica de evaluacion

Cuando se trabaja con vectores binarios (es decir, con valores de 0 o 1), la similitud del coseno se convierte en una medida de la relación direccional entre los vectores, en lugar de una medida de magnitud. Si dos vectores binarios tienen valores similares en las mismas posiciones, su similitud del coseno será mayor. Por otro lado, si los vectores difieren significativamente en las mismas posiciones, la similitud del coseno será menor.

Para calcular la similitud del coseno entre vectores binarios, se considera que los valores de 1 indican la presencia de una característica o atributo, mientras que los valores de 0 indican la ausencia de la misma. La similitud del coseno se calcula basándose en el número de características que comparten dos vectores y en la dirección relativa de esas características

In [160]:
from sklearn.metrics.pairwise import cosine_similarity

# Define la función para calcular la similitud del coseno entre dos conjuntos de vectores
def compute_cosine_similarity(predictions, labels):
    # Calcula la similitud del coseno entre las predicciones y las etiquetas
    similarity_matrix = cosine_similarity(predictions, labels)
    # La similitud del coseno entre dos vectores es el valor en la diagonal de la matriz de similitud
    mean_cosine_similarity = np.mean(np.diag(similarity_matrix))
    return mean_cosine_similarity

# Define la función de evaluación de métricas
def compute_metrics(p):
    predictions, labels = p
    # Utiliza la función para calcular la similitud del coseno entre las predicciones y las etiquetas
    
    print("Aqui va el modelo")
    print('imprime p:')
    print(p)
    print("Imprime predictions:")
    print(predictions)
    print(predictions.shape)
    print('labels:')
    print(labels)
    print(labels.shape)
    # re dimensionamiento de predictions
    predictions_1d = np.max(predictions, axis=1).reshape(-1, 1)
    print('predictions reshape')
    print(predictions_1d)
    cosine_sim = compute_cosine_similarity(predictions_1d, labels.reshape(-1, 1))
    print('paso el calculo')
    return {"cosine_similarity": cosine_sim}

Modelo 1 - BASELINE de entrenamiento
Esta configuracion se basa en el articulo:

loRA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS - 2021 Edward Hu∗ Yelong Shen∗ Phillip Wallis Zeyuan Allen-Zhu Yuanzhi Li Shean Wang Lu Wang Weizhu Chen

Una red neuronal contiene muchas capas densas que realizan la multiplicación de matrices. Las matrices de peso en estas capas suelen tener rango completo. Al adaptarse a una tarea específica, muestra que los modelos de lenguaje previamente entrenados tienen una "dimensión instrinseca" baja y aún pueden aprender de manera eficiente a pesar de una proyeccion aleatoria a un subespacio más pequeño. A raiz de esto, se plantea la hipotesis de que las actualizaciones de las ponderaciones también tienen un "rango intrínseco" bajo durante la adaptacion.

Al aplicar LoRA a todas las matrices de peso y entrenar todos los sesgos, recuperamos aproximadamente la expresividad del ajuste fino completo al establecer el rango r de LoRA en el rango de las matrices de peso previamente entrenadas. En otras palabras, a medida que aumentamos la cantidad de parámetros entrenables, el entrenamiento de LoRA converge aproximadamente al entrenamiento del modelo original.

LoRA se basa en truncar la descomposicion de valores singulares (SVD) de una matriz de rango superior para obtener la aproximacion. Es posible encontrar una aproximación de rango reducida a A estableciendo todos, excepto los primeros k valores singulares más grandes, iguales a cero y usando solo las primeras k columnas de U y V, en la ecuación: A = U * S * V(T). Ademas, de reemplazar por ser los valores singulares mas pequeños en la diagonal de S. Despues de un fine tunning, las matrices de rango bajo se pueden reconstruir en matrices de rango completo usando la misma tecnica de descomposición, y el modelo de lenguaje grande (LLM) adaptado resultante, se puede usar para tareas posteriores en el nuevo dominio.

LoRA es el proceso de representar la información en una Matriz M utilizando una matriz M2 que tiene un rango menor que la matriz original. Para reducir el rango de M2, podemos intentar construir la matriz como una combinación de una matriz izquierda Lk y una matriz amplia derecha Rk transpuesta.

Por ejemlpo para una capa de una red neuronal se tiene:

h(x) = W0 * x + delta W * x siendo : delta W = BA Entonces:

h(x) = W0 * x + B * A * X

DONDE :

W0 pertenece a todo R elevado a (d x k) B pertenece a todo R elevado a (d x r) A pertecene a todo R elevado a (r x k) h pertenece a todo R elevado a (d x 1)

siendo r, un factor mutiplicador que define el tamaño de la matrix B y A que corresponden a los pesos que seran entrenados en la red neuronal y dejando congelado el peso de W0. Este valor de r es el que se ve en la siguiente linea de codigo r = 4. Hay un termino adicional alpha que multiplica a: alphaBA*x que se traduce como un para la reparametrizacion de LoRA.

Algunas ventajas de esta tecnica son:

la reduccion de ruido
compresion de las matrices para reducir el gaste computacional
detecta patrones escondidos en los datos.
Parameter efficient fine tunning -LoRA (Low Rank Adaptation) una proyeccion aleatoria a un pequeño subespacio
LoRA: congela los pesos de los modelos previamente entrenados e introduce matrices de descomposicion de rangos entrenables en cada capa del transformer.
Fine tunning implica copiar los pesos de la red previamente entrenada y ajustarlos en la tarea posterior (un nuevo conjunto de pesos de cada tarea).
LoRA reduce el numero de paramtros entrenables por 10,000 veces y los requerimientos de la memoria de la GPU por 3 veces.

In [161]:
peft_config = LoraConfig(task_type="SEQ_CLS", # sequence classification
                        r=4, # intrinsic rank of trainable weight matrix
                        lora_alpha=16, # used for scalling the LoRA reparametrization .
                        lora_dropout=0.01, # probablity of dropout
                        target_modules = ['q_lin']) # we apply LoRA to query layer only

In [162]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters() 

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


In [163]:
# hyperparametros
lr = 1e-3 # size of optimization step 
batch_size = 4 # number of examples processed per optimziation step
num_epochs = 2 # number of times model runs through training data

# define training arguments
training_args = TrainingArguments(
    output_dir= modelo + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    dataloader_num_workers = 4,
    dataloader_prefetch_factor=2
)

In [164]:
# creater trainer object
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to('cuda')
trainer = Trainer(
    model=model, # our peft model
    args=training_args, # hyperparameters
    train_dataset= dataset_dict['train'], # training data 
    eval_dataset=dataset_dict['test'], # test data  
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics, # evaluates model using compute_metrics() function from before
)

# train model
trainer.train()


 50%|█████     | 1000/2000 [08:08<08:08,  2.05it/s]






































































































                                                    

 50%|█████     | 1000/2000 [02:38<00:21, 46.77it/s]




{'loss': 0.4192, 'grad_norm': 0.723501980304718, 'learning_rate': 0.00075, 'epoch': 0.5}

































































































                                                    

 50%|█████     | 1000/2000 [02:49<00:21, 46.77it/s]


{'loss': 0.4167, 'grad_norm': 0.348330020904541, 'learning_rate': 0.0005, 'epoch': 1.0}






































































































































































                                                    

                                             


 50%|█████     | 1000/2000 [03:25<00:21, 46.77it/s]




Checkpoint destination directory distilbert-base-uncased-lora-text-classification\checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


...


































































































                                                    

 50%|█████     | 1000/2000 [04:13<00:21, 46.77it/s]


{'loss': 0.4051, 'grad_norm': 0.438629686832428, 'learning_rate': 0.0, 'epoch': 2.0}






































































































































































                                                    

                                             


 50%|█████     | 1000/2000 [04:48<00:21, 46.77it/s]




Checkpoint destination directory distilbert-base-uncased-lora-text-classification\checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.

                                                    

100%|██████████| 2000/2000 [02:47<00:00, 11.97it/s]

Aqui va el modelo
imprime p:
Imprime predictions:
[[ 1.8892407 -1.8632411]
 [ 1.9151245 -1.8756496]
 [ 1.8987005 -1.8726971]
 ...
 [ 1.8910749 -1.8682263]
 [ 1.9762089 -1.9427791]
 [ 1.9032049 -1.8801205]]
(4000, 2)
labels:
[0 0 0 ... 0 0 1]
(4000,)
predictions reshape
[[1.8892407]
 [1.9151245]
 [1.8987005]
 ...
 [1.8910749]
 [1.9762089]
 [1.9032049]]
paso el calculo
{'eval_loss': 0.40097010135650635, 'eval_cosine_similarity': 0.1, 'eval_runtime': 35.0092, 'eval_samples_per_second': 114.256, 'eval_steps_per_second': 28.564, 'epoch': 2.0}
{'train_runtime': 167.1156, 'train_samples_per_second': 47.871, 'train_steps_per_second': 11.968, 'train_loss': 0.4112116851806641, 'epoch': 2.0}


TrainOutput(global_step=2000, training_loss=0.4112116851806641, metrics={'train_runtime': 167.1156, 'train_samples_per_second': 47.871, 'train_steps_per_second': 11.968, 'train_loss': 0.4112116851806641, 'epoch': 2.0})

Se puede observar un ejemplo de nuestro nuevo modelo de como clasifica un texto. 

In [165]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Trained model predictions:")
print("--------------------------")
for text in train_tokenized[['text']].iloc[1]:  
    # Tokeniza y codifica el texto
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)

    # Realiza la inferencia en el modelo
    logits = model(inputs).logits
    predictions = torch.max(logits, 1).indices
    print(label1[predictions.tolist()[0]]) 

    print(text + " - " + label1[predictions.tolist()[0]])

Trained model predictions:
--------------------------
Norelevant
Sprint pagar por teléfono número de teléfono 1 El engranaje saca un teléfono. 2 El engranaje dice una frase. 3 El engranaje marca un número en el teléfono. A medida que el engranaje presiona el número, los signos de almohadilla (#) saldrán del teléfono y atacarán al personaje objetivo, haciéndolo temblar. - Norelevant


In [166]:
# realizamos las predicciones para los datos de testeo para calcular las metricas y ver la calidad del modelo con los datos utilizados
prediction_f = []
for index, row in train_tokenized.iterrows(): 
    text = row['text']

    # Tokeniza y codifica el texto
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)

    # Realiza la inferencia en el modelo
    logits = model(inputs).logits
    predictions = torch.max(logits, 1).indices
    prediction_f.append([text, predictions.tolist()[0]])
if len(prediction_f) > 0:
    result = pd.DataFrame(prediction_f).rename(columns={0:'text',1:'label_predicted'})

In [168]:
from sklearn.metrics import accuracy_score, confusion_matrix


accuracy = accuracy_score(train_tokenized['label'], result['label_predicted'])
print(f'Accuracy: {accuracy:.2f}')

# Calcula la matriz de confusión
conf_matrix = confusion_matrix(train_tokenized['label'], result['label_predicted'])
print('Confusion Matrix:')
print(conf_matrix) 

Accuracy: 0.90
Confusion Matrix:
[[3600    0]
 [ 400    0]]


A pesar de que el accuracy es alto, se observa que la matrix de confusion presenta varios falsos negativos. Es decir que no es capaz de distinguir que texto no es importante, y todo lo toma como importante. 

In [169]:
model.eval() # se pone el modelo en modo evaluacion 

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): PeftModelForSequenceClassification(
      (base_model): LoraModel(
        (model): PeftModelForSequenceClassification(
          (base_model): LoraModel(
            (model): PeftModelForSequenceClassification(
              (base_model): LoraModel(
                (model): PeftModelForSequenceClassification(
                  (base_model): LoraModel(
                    (model): PeftModelForSequenceClassification(
                      (base_model): LoraModel(
                        (model): PeftModelForSequenceClassification(
                          (base_model): LoraModel(
                            (model): PeftModelForSequenceClassification(
                              (base_model): LoraModel(
                                (model): PeftModelForSequenceClassification(
                                  (base_model): LoraModel(
                                    (model): DistilBertForSequenceClassif

In [170]:
# Se crea el diccionario con la informacion de input y attention mask de documentos y de query.
data_test_documents = {
    'input_ids': torch.tensor(input_test_documents),
    'attention_mask': torch.tensor(attention_mask_test_documents)
}

data_test_query = {
    'input_ids': torch.tensor(input_test_queries),
    'attention_mask': torch.tensor(attention_mask_test_queries)
}

In [171]:
model.to('cpu') 

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): PeftModelForSequenceClassification(
      (base_model): LoraModel(
        (model): PeftModelForSequenceClassification(
          (base_model): LoraModel(
            (model): PeftModelForSequenceClassification(
              (base_model): LoraModel(
                (model): PeftModelForSequenceClassification(
                  (base_model): LoraModel(
                    (model): PeftModelForSequenceClassification(
                      (base_model): LoraModel(
                        (model): PeftModelForSequenceClassification(
                          (base_model): LoraModel(
                            (model): PeftModelForSequenceClassification(
                              (base_model): LoraModel(
                                (model): PeftModelForSequenceClassification(
                                  (base_model): LoraModel(
                                    (model): DistilBertForSequenceClassif

In [172]:
# obtenemos el tensor de salida de nuestro modelo para docuemntos y para las consultas y de esta manera calcular la similitud coseno. 
output_document = model(**data_test_documents) 
output_query = model(**data_test_query) 

In [173]:
tensor1_ = output_query.logits
tensor2_ = output_document.logits

consulta_y_documentos_similares_id = []
consulta_y_documentos_similares = []

for i in range(tensor1_.size(0)):
# Calcular la similitud coseno entre la consulta y cada documento
    similarities_ = util.cos_sim(tensor1_[i], tensor2_)
    word_query = test_q['query'][i]
    word_query_id = test_q['query_id'][i]
    # Obtener los índices de los documentos más similares (en orden descendente)
    top_document_indices_ = similarities_.argsort(descending=True)[0][:10] 
    # Imprimir los documentos más similares para la consulta actual
    #print("\nConsulta:",tensor_queries_word[i]) 
    #print("Documentos más similares:")

    documentos_similares_id = []
    documentos_similares = []
    for index in range(10):
        indice = top_document_indices_.tolist()[index]
        documento_similar_id = test_d['document_id'][indice] 
        documento_similar = test_d['document'][indice] 
        documentos_similares_id.append(documento_similar_id) 
        documentos_similares.append(documento_similar) 
    consulta_y_documentos_similares_id.append({'query_id': word_query_id, 'document_id': documentos_similares_id})
    consulta_y_documentos_similares.append({'query': word_query, 'document': documentos_similares})


In [174]:
consulta_y_documentos_similares

[{'query': '¿Cuánto tiempo dura el proceso de una prueba de paternidad?',
  'document': ['La ley de renuncia y su definición legal se refiere a la situación en la cual un padre biológico decide voluntariamente renunciar a sus derechos parentales sobre un niño, con el propósito de permitir que el niño sea adoptado.',
   'Estoy buscando una lámpara de mesa grande con un cuello de cisne hecho de mosaico turco. Es un modelo de ensueño del tiempo otomano y está hecho completamente a mano.',
   'El condado de Rockingham, ubicado en el estado de Virginia, es un lugar donde puedes encontrar automóviles para comprar. Su asiento de condado es Harrisonburg y en el censo de 2010 tenía una población de 76,314 habitantes.',
   'Es crucial que el apéndice sea removido antes de que se rompa. La extirpación del apéndice, conocida como apendicectomía, puede realizarse mediante una incisión de dos o tres pulgadas en la parte inferior derecha del abdomen, o a través de una cirugía laparoscópica.',
   'El 

In [175]:
# Crear un diccionario para almacenar los resultados
results_dict_id = {}

# Procesar cada consulta y encontrar los documentos más similares
for i in range(len(consulta_y_documentos_similares_id)):
    # Obtener el word_query_id correspondiente a esta consulta
    word_query_id = consulta_y_documentos_similares_id[i]['query_id']
    # Agregar los IDs de los documentos al diccionario
    results_dict_id[word_query_id] = consulta_y_documentos_similares_id[i]['document_id']

# Convertir el diccionario a formato JSON
import json
json_results_id = json.dumps(results_dict_id, indent=4)

# Imprimir los resultados JSON
print(json_results_id)

{
    "KWjLrafk": [
        "RR70GoQP",
        "SIm9LzFq",
        "bsSl9IHF",
        "mgknjCxs",
        "bt02NCh4",
        "S251l5UU",
        "MdNkZEPo",
        "8dnCgV8g",
        "4nggpl7e",
        "EOImsiBu"
    ],
    "S26n47qu": [
        "Hr4WQM4b",
        "2vUOadAm",
        "DjV1jzoW",
        "9TvbhKcb",
        "7xETR9If",
        "i4kyAxae",
        "zTydcHYL",
        "jGkNsRNB",
        "HtRH7pCY",
        "gv4dhAKa"
    ],
    "Jj7Rmaeu": [
        "Sk782pBA",
        "u6CkqWgE",
        "mgknjCxs",
        "6YEO7wDf",
        "uYH4hhEl",
        "r3VP7EiW",
        "UiwKqvqk",
        "h9K22P3d",
        "eKPB5mg8",
        "6GotmtuF"
    ],
    "XxxdpKqh": [
        "l8EruhOZ",
        "ESvEcg9G",
        "HZNz5K7x",
        "e9pGi8bd",
        "BjWdlwLr",
        "uNhvTTsS",
        "UiwKqvqk",
        "4u5VbM7v",
        "aYfiF8Qw",
        "hsWugY4R"
    ],
    "Ix7Dq5ey": [
        "GqNjDbMX",
        "4qdkPqSM",
        "Ma6nw2WQ",
        "VZ8Ui042",
    

In [180]:
# guardamos los resultados
ruta_archivo = 'resultado_clasificador.json'

# Guarda los datos en el archivo JSON
with open(ruta_archivo, 'w') as archivo:
    json.dump(results_dict_id , archivo) 